In [1]:
import pandas as pd
import numpy as np
import emoji
import re
from emot.emo_unicode import EMOTICONS_EMO
from nltk.tokenize import ToktokTokenizer

In [5]:
#loading the training dataset
train = pd.read_csv('Dataset/tamil_offensive_full_train.csv', delimiter="\t", error_bad_lines=False, header=None, 
                    names=['text', 'label', 'nan'])
train = train.drop(columns=['nan'])
train.dropna()
train['label'] = pd.Categorical(train.label)
train

C:\Users\Admin\AppData\Local\Temp/ipykernel_8596/3892080247.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train = pd.read_csv('Dataset/tamil_offensive_full_train.csv', delimiter="\t", error_bad_lines=False, header=None,


,text,label
0,movie vara level la Erika poguthu,Not_offensive
1,I love Ajith Kumar Vivegam movie inki mjy bht ...,not-Tamil
2,Padam nalla comedy padama irukum polaye..,Not_offensive
3,karthick subburaj anne .... intha padam vetri ...,Not_offensive
4,கவுண்டர் தேவர்.சார்பாக வெற்றி பெற வாழ்த்துக்கள் 🦁,Not_offensive
...,...,...
35134,Trending number #2 idhukku nammalam karanamnu ...,Not_offensive
35135,"Movie script super, athuvum HIP HOP Tamizha mu...",Not_offensive
35136,Just 3k likes for 300k likes,Not_offensive
35137,Aaloo le lo. Kanda le lo.,not-Tamil


In [6]:
val = pd.read_csv('Dataset/tamil_offensive_full_dev.csv', delimiter="\t", error_bad_lines=False, header=None, 
                  names=['text', 'label', 'nan'])
val = val.drop(columns=['nan'])
val.dropna()
val['label'] = pd.Categorical(val.label)
val

C:\Users\Admin\AppData\Local\Temp/ipykernel_8596/3735339048.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  val = pd.read_csv('Dataset/tamil_offensive_full_dev.csv', delimiter="\t", error_bad_lines=False, header=None,


,text,label
0,Handsome hunk keri vaa thalaivaa,Not_offensive
1,தென்காசி மாவட்டம் நாடார் சமுதாயம் சார்பாக வாழ்...,Not_offensive
2,je vous aime bravo pour clip de merde que j éc...,not-Tamil
3,சிறப்பு..... மேலும் இது போன்ற படைப்புகள் மிக அ...,Not_offensive
4,Vera level BGM .. semma trailer. 🤞,Not_offensive
...,...,...
4383,மிஷ்கின் - சினிமாவின் சலித்து போன வட்டத்திற்க...,Not_offensive
4384,Sivaji - Bhajii Sapdu Petta - Sweet Sapdu Ser...,Not_offensive
4385,8k dislike sure all vijay fans,Offensive_Targeted_Insult_Other
4386,Lady super star Manju warrier Fans Hit like,Not_offensive


In [11]:
def deEmojify(string):
    emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                              u"\U00002500-\U00002BEF"  # chinese char
                              u"\U00002702-\U000027B0"
                              u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"
                              u"\U0001f926-\U0001f937"
                              u"\U00010000-\U0010ffff"
                              u"\u2640-\u2642"
                              u"\u2600-\u2B55"
                              u"\u200d"
                              u"\u23cf"
                              u"\u23e9"
                              u"\u231a"
                              u"\ufe0f"  # dingbats
                              u"\u3030"
                              "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
        
def preprocess(text):
    text = str(text)
    text = deEmojify(text) #removing emojis
    text = re.sub(r'([\.\'\"\/\-\_\--])',' ', text) # remove punctuations , removes @USER / some abbreviatins
    to_remove_url = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
      '[!*,]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = re.sub(to_remove_url,'',text)  # remove url patterns
    text = re.sub(" \d+", " ", text)
    text = text.replace(","," ")
    text = re.sub(r'(?:^| )\w(?:$| )', ' ', text).strip()
    punctuation='!!"$%&()*+-/:;<=>?[\]^_{|}~.'
    text = ''.join(ch for ch in text if ch not in set(punctuation))
    
    tokenizer = ToktokTokenizer()
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    text = ' '.join(ch for ch in tokens)
    return text 

def clean(df):
    df['text'] = df['text'].apply(lambda x: preprocess(x))

clean(train)
clean(val)

In [2]:
from transformers import MBartForConditionalGeneration
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")

from transformers import MBart50TokenizerFast
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")
article_en = "movie super, nalla movie"
model_inputs = tokenizer(article_en, return_tensors="pt")
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["ta_IN"]
)
translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

translation[0]

C:\Users\Admin\anaconda3\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


C:\Users\Admin\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'படம் சூப்பர், நல்ல படம்'

In [3]:
def transliterate(text):
    model_inputs = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["ta_IN"]
    )
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return translation[0]

In [17]:
train['text'] = train['text'].apply(lambda x: transliterate(x))
train.to_csv('tamil_train_translit_v2.csv')
val['text'] = val['text'].apply(lambda x: transliterate(x))
val.to_csv('tamil_dev_translit_v2.csv')

['திரைப்படம் தரம் லா எரிகா படப்பிடிப்பு']
['காதல் அஜித் குமார் விவேகம் படம் inki mjy bht achi lgi']
['படம் நல்ல காமெடி படம் இனிமை போலே']
['கர்திக் சுப்பூராஜ் அன்னை இன்தா பதம் வேத்ரி அதயா ஒருகளுக்கு என்னுடய வள்ளுக்கல்']
['கவுண்டர் தேவர் சார்பாக வெற்றி பெற வாழ்த்துக்கள்']
['ஐப்போ இன்தா டிரைலர் ஆ பார்குரவன் ஓ ஓ ஒடுங்காவைப்போல்.']


KeyboardInterrupt: 

In [28]:
transliterate("Movie script super, HIP HOP tamizha")

'படம் ஸ்கிரிப்ட் சூப்பர், ஹீப் ஹோப் தமிழா'